# Data analysis in Python using built-in types

Date: June 22, 2019

-----

## Learning Objectives

* Using the command line to:
    + make directories
    + download files from internet
    + unzip `.zip` files
* Import a `.csv` file without [`pandas`](https://pandas.pydata.org/about.html) or [`numpy`](https://www.numpy.org/)
* Perform data analysis using built-in types
    + working with dates
    + sorting dictionaries
    + for loops and list comprehension
    + visualizing data with [`matplotlib`](https://matplotlib.org/index.html)

## Inspect Files in current working directory

In [1]:
!ls

LICENSE             data_analysis.ipynb raw_data
README.md           hello_world         visuals


## Create a new `raw_data` directory

In [2]:
!mkdir raw_data

mkdir: raw_data: File exists


Let's reinspect our working directory. By typing `ls -l`, we are getting more information about all files and folders such as when things were created.

In [3]:
!ls -l

total 160
-rw-r--r--  1 DirkStrong1  staff  35149 Aug  9 09:34 LICENSE
-rw-r--r--  1 DirkStrong1  staff    543 Aug  9 09:34 README.md
-rw-r--r--  1 DirkStrong1  staff  34600 Aug  9 11:13 data_analysis.ipynb
-rw-r--r--  1 DirkStrong1  staff     10 Aug  9 10:01 hello_world
drwxr-xr-x  4 DirkStrong1  staff    136 Aug  9 10:45 raw_data
drwxr-xr-x  2 DirkStrong1  staff     68 Aug  9 10:59 visuals


## Install Homebrew

[`homebrew`](https://brew.sh/) is the package manager for MacOS. This tutorial assumes that you have it installed on your machine.

*If not, please uncomment and run the cell below to install `homebrew`.*

In [4]:
#/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"

## Download necessary data

Let's download [King County, WA property sales data](https://info.kingcounty.gov/assessor/DataDownload/default.aspx) into our newly created `raw_data` directory. We'll be using the [`wget`](https://www.gnu.org/software/wget/) package to download local copies of files from the Internet. 

The default directory is the current working directory so we need to specify where `wget` should place the file via the `-P` flag.

```
# <> are placeholders so do not include them in your commands
wget <url> -P <file path>
```

*If you don't have `wget`, uncomment the cell below to have `brew` install it for you.*

In [5]:
!brew install wget

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/cask).
No changes to formulae.

To reinstall 1.20.3_1, run `brew reinstall wget`


In [6]:
!wget https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip -P raw_data/

--2019-08-09 13:57:58--  https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip
Resolving aqua.kingcounty.gov (aqua.kingcounty.gov)... 146.129.240.28
Connecting to aqua.kingcounty.gov (aqua.kingcounty.gov)|146.129.240.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125002754 (119M) [application/x-zip-compressed]
Saving to: ‘raw_data/Real Property Sales.zip.1’

Real Property Sales 100%[===================>] 119.21M  10.6MB/s    in 12s     

2019-08-09 13:58:10 (10.2 MB/s) - ‘raw_data/Real Property Sales.zip.1’ saved [125002754/125002754]



Let's inspect the contents of `raw_data` to verify that we have the data

In [7]:
!ls -l raw_data/

total 1542904
-rw-r--r--  1 DirkStrong1  staff  539953127 Aug  2 18:13 EXTR_RPSale.csv
-rw-r--r--  1 DirkStrong1  staff  125002754 Aug  3 00:30 Real Property Sales.zip
-rw-r--r--  1 DirkStrong1  staff  125002754 Aug  3 00:30 Real Property Sales.zip.1


### Unzip `raw_data/Real Property Sales.zip`

Currently the data lives in a [`.zip`](http://cvs.bio.unc.edu/protocol/unzip-help/unzip-help.htm) file, which is essentially a layer that contains the actual data.

To extract - or unpeel - the contents within the `.zip` file, we'll use the [`unzip`](https://formulae.brew.sh/formula/unzip) command. 

By specifying `-d raw_data/`, we're telling `unzip` to place the contents of `raw_data/Real Property Sales.zip` inside the `raw_data/` directory.

*If you don't have `unzip`, uncomment the cell below to have `brew` install it for you.*

In [8]:
!brew install unzip

To reinstall 6.0_5, run `brew reinstall unzip`


In [ ]:
!unzip raw_data/Real\ Property\ Sales.zip -d raw_data/

Archive:  raw_data/Real Property Sales.zip
replace raw_data/EXTR_RPSale.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

After unzipping `raw_data/Real Property Sales.zip`, we now have access to the `raw_data/EXTR_RPSale.csv`.

In [ ]:
!ls -l raw_data/

## Load necessary data

> The `csv` module implements classes to read and write tabular data in CSV format. It allows programmers to say, “write this data in the format preferred by Excel,” or “read data from this file which was generated by Excel,” without knowing the precise details of the CSV format used by Excel. - [`csv` documentation](https://docs.python.org/3/library/csv.html)

### Using the `csv` library

The `csv` library contains objects and other code to read, write, and process data from and to CSV files. Essentially, we want to read each row in the `.csv` file and stores its information for later use. 


### How `csv` works

The [`csv.DictReader()`](https://docs.python.org/3/library/csv.html#csv.DictReader) function creates an object that operates like a regular reader - where information is returned as [strings](https://docs.python.org/3.7/library/stdtypes.html#text-sequence-type-str) - but maps the information in each row to an [OrderedDict](https://docs.python.org/3/library/collections.html#collections.OrderedDict) whose keys are given by the optional `fieldnames` parameter. If `fieldnames` is omitted, the values in the first row of file f will be used as the fieldnames.

Since each row is an `OrderedDict` object, we know that information in each dictionary will be in the same order as they appear within the `.csv` file.

### Inspect the first two rows in the `raw_data/EXTR_RPSale.csv` file

In [ ]:
!head -n 2 raw_data/EXTR_RPSale.csv

Now that we know that the first row is the column headers, we can safely omit the `fieldnames` parameter knowing that `csv.DictReader()` will use the first row as the keys within each row.

In [ ]:
import csv

sales = []
with open('raw_data/EXTR_RPSale.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        sales.append(row)

The contents of `sales[0]` is exactly the same as it was when we viewed the first two records of the `raw_data/EXTR_RPSale.csv` file from the command line.

In [ ]:
sales[0]

## Data Analysis

Now that you have all the property sales data from King County, WA within the list `sales`, you can finally start to perform some data analysis.

### 1. For each year, how many houses were sold?

Anytime you encounter questions regarding time, you need to know if your data is in [`datetime`](https://docs.python.org/3.7/library/datetime.html) format. `datetime` objects make it easy to date and time calculation.

In [3]:
from datetime import datetime

In [4]:
for sale in sales:
    # transform DocumentDate from string to datetime
    sale["clean_date"] = datetime.strptime(sale["DocumentDate"], 
                                           "%m/%d/%Y")
    # store the sale year and month
    sale["sale_year"] = sale["clean_date"].year
    sale["sale_month"] = sale["clean_date"].month

Let's inspect our work

In [5]:
sales[0]

OrderedDict([('ExciseTaxNbr', '2687551'),
             ('Major', '138860'),
             ('Minor', '0110'),
             ('DocumentDate', '08/21/2014'),
             ('SalePrice', '245000'),
             ('RecordingNbr', '20140828001436'),
             ('Volume', '   '),
             ('Page', '   '),
             ('PlatNbr', '      '),
             ('PlatType', ' '),
             ('PlatLot', '              '),
             ('PlatBlock', '       '),
             ('SellerName',
              'WENKLE NOEL SMITH -TRUSTEE                        '),
             ('BuyerName',
              'ALEXANDER APRIL                                   '),
             ('PropertyType', '3'),
             ('PrincipalUse', '6'),
             ('SaleInstrument', '3'),
             ('AFForestLand', 'N'),
             ('AFCurrentUseLand', 'N'),
             ('AFNonProfitUse', 'N'),
             ('AFHistoricProperty', 'N'),
             ('SaleReason', '1'),
             ('PropertyClass', '8'),
             ('Sa

For all dictionaries in `sales`, we want to count how many property sales occured in each year. Since the value in `sale_year` is an integer, we can used Python's built-in [`sorted`](https://docs.python.org/3/library/functions.html#sorted) function to transform the `Counter` object into a `list` of `tuples` - `(year, count)` which are sorted in ascending order by year.

In [6]:
from collections import Counter

sale_year_count = Counter([sale['sale_year'] for sale in sales])
sale_year_count = sorted(sale_year_count.items())
sale_year_count

[(1900, 1),
 (1934, 1),
 (1951, 1),
 (1953, 1),
 (1954, 9),
 (1955, 3),
 (1956, 1),
 (1957, 3),
 (1958, 1),
 (1959, 4),
 (1960, 10),
 (1961, 5),
 (1962, 5),
 (1963, 9),
 (1964, 20),
 (1965, 11),
 (1966, 12),
 (1967, 18),
 (1968, 9),
 (1969, 20),
 (1970, 17),
 (1971, 14),
 (1972, 27),
 (1973, 45),
 (1974, 35),
 (1975, 43),
 (1976, 62),
 (1977, 97),
 (1978, 103),
 (1979, 124),
 (1980, 186),
 (1981, 408),
 (1982, 6035),
 (1983, 8607),
 (1984, 9199),
 (1985, 10557),
 (1986, 14627),
 (1987, 13575),
 (1988, 14190),
 (1989, 17232),
 (1990, 16606),
 (1991, 17524),
 (1992, 67859),
 (1993, 69456),
 (1994, 64912),
 (1995, 57982),
 (1996, 63652),
 (1997, 72090),
 (1998, 79578),
 (1999, 75237),
 (2000, 69874),
 (2001, 71260),
 (2002, 77031),
 (2003, 86299),
 (2004, 89383),
 (2005, 93483),
 (2006, 84728),
 (2007, 73246),
 (2008, 51802),
 (2009, 52384),
 (2010, 54678),
 (2011, 56177),
 (2012, 62440),
 (2013, 69396),
 (2014, 64251),
 (2015, 68893),
 (2016, 70399),
 (2017, 68371),
 (2018, 60944),
 (201

This `list` of `tuples` is not something our employers can use to make a decision. Before we visualize the results, let's create a new `visuals` folder that will store all of our visuals.

In [5]:
!mkdir visuals

mkdir: visuals: File exists


In [6]:
!conda install matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [ ]:
!conda update -n base -c defaults conda

Solving environment: done

## Package Plan ##

  environment location: //anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.11               |           py37_0         3.0 MB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  conda                                       4.7.10-py37_0 --> 4.7.11-py37_0


Proceed ([y]/n)? 

In [ ]:
y

In [7]:
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
plt.bar(x=[sale[0] for sale in sale_year_count],
        height=[sale[1] for sale in sale_year_count])
plt.xlabel("Year")
plt.ylabel("Number of property sales")
plt.title("Property sales in King County, WA (1900 to 2021)")
plt.savefig("visuals/raw_sales_count_by_year.png",
            dpi=150)
plt.show()

NameError: name 'plt' is not defined

By the looks of our number of property sales over time bar plot, it looks like property sales in King County, WA started taking off in the 1980s. They continued climbing into the early 2000s. Despite the recession reducing the number of home sales in the late 2000s, it looks like they are continuing to climb.

### 2. After removing the one record that took place in 2021, what is the annual average [nominal](https://www.stlouisfed.org/publications/inside-the-vault/fall-2007/nominal-vs-real-oil-prices) property sales price in King County, WA?

### 3. Which property has been sold the most in King County, WA throughout time?

This questions requires you to create a <strong>P</strong>arcel <strong>I</strong>dentification <strong>N</strong>umber (PIN). To learn how to do this, read the "Attribute Information" section on the Parcel webpage from the [King County Geographic Information System (KCGI)](https://www5.kingcounty.gov/sdc/Metadata.aspx?Layer=parcel).

### 4. Who are the top 5 sellers in terms of who has sold the most property in King County, WA throughout time?

### 5. From 2009 to 2019, which buyer has bought the most property in King County, WA?